## **SpDB Manual之系列4：MDSplus数据库系统**

#### **SpDB**处理远程MDSplus数据库系统
在**SpDB**中，使用open_entry()建立统一的访问入口,支持远程MDSplus Server数据库的访问，及本地MDSplus的数据库中数据的访问。支持两个有用的功能：

（1）**不同数据源的自动集成**

SpDB中提供**open_entry**建立访问链接，**wall，pf，tf，magnetics**等被映射的数据均可以通过该链接入口访问。数据源来自于静态的XML文件，动态的MDS数据库中的不同tree：east,pcs_east,efit_east等

SpDB中数据的访问方式是按照IDS的树状结构逐层访问。

（2）**“懒惰加载”数据**

SpDB支持”懒惰加载“数据，因为，SpDB后台已经自动集成了不同数据源的数据，对应于不同的IDS条目中。所请求的条目或者某个条目中可能存储了大量的数据，如果请求的时候便立即从底层访问层后端读取所有数据，可能需要很长时间才能完成。通常情况下，用户可能只需要个别的数据子集，”懒惰加载”使得用户仅仅建立链接，只有在需要的时候才真正获取数据，这样会利于加速。

- **.child()**操作可以将链接指针移动到下一个树节点，建立新的链接，而不会获取数据
- **.get()**操作则直接将获得全部数据。

#### 加载基本环境

In [3]:
### import基本环境
from spdm.data.Entry import open_entry
from spdm.utils.logger import logger
# import fytok
import MDSplus
import os

### 指定mappingfile的路径
# os.environ["SP_DATA_MAPPING_PATH"] = "${workdir}/fytok_data/mapping"

In [17]:
os.environ["SP_DATA_MAPPING_PATH"] = "/gpfs/fuyun/projects/fuyun/fytok/python/fytok/_mapping"

#### 访问远程EAST MDSplus数据库

In [18]:
### 访问远程EAST MDS数据库中70754炮的数据，
shot_num = 70754
time_slice = 10
entry = open_entry("east+mdsplus://202.127.204.12?enable=efit_east&shot={shot_num}")

FileNotFoundError: Can not find mapping files for ['east', 'imas/3', 'mdsplus'] MAPPING_PATH=[] !

#### 访问远程本地MDSplus数据库

In [19]:
# ### 访问本地MDSplus路径中中的70754炮的数据
shot_num = 70754
time_slice = 10
DATA_PATH = "/scratch/jupytertest/workspace_fytok/fytok_data"
entry = open_entry(f"east+mdsplus://{DATA_PATH}/mdsplus/~t/?shot=70745")

FileNotFoundError: Can not find mapping files for ['east', 'imas/3', 'mdsplus'] MAPPING_PATH=[] !

#### .child()支持“懒惰执行”

In [417]:
### .child操作会将链接进一步指向wall
wall = entry.child("wall")
### 打印wall的类型，仍然是个entry
type(wall)

spdm.data.Entry.EntryProxy

In [418]:
### 继续移动链接到wall中的下一个子节点中,outline仍然是个entry
outline = entry.child("wall.description_2d[0].limiter.unit[0].outline")
### 获取outline的数据
print(outline.__value__)

{'r': array([2.277, 2.273, 2.267, 1.94 , 1.94 , 1.802, 1.773, 1.751, 1.736,
       1.714, 1.707, 1.696, 1.665, 1.656, 1.635, 1.612, 1.478, 1.459,
       1.44 , 1.436, 1.399, 1.379, 1.392, 1.43 , 1.439, 1.442, 1.437,
       1.363, 1.361, 1.361, 1.361, 1.363, 1.421, 1.423, 1.422, 1.418,
       1.331, 1.367, 1.564, 1.597, 1.598, 1.624, 1.754, 1.765, 1.814,
       1.824, 1.825, 1.841, 1.971, 1.971, 2.267, 2.273, 2.277, 2.277,
       2.306, 2.328, 2.343, 2.35 , 2.35 , 2.35 , 2.343, 2.328, 2.306,
       2.277]), 'z': array([ 0.485,  0.485,  0.493,  0.809,  0.809,  0.926,  0.956,  0.993,
        1.033,  1.131,  1.162,  1.142,  1.117,  1.111,  1.096,  1.084,
        1.025,  1.021,  1.024,  1.026,  1.039,  1.049,  1.014,  0.909,
        0.873,  0.835,  0.799,  0.456,  0.454,  0.   , -0.454, -0.456,
       -0.725, -0.748, -0.749, -0.77 , -1.011, -0.977, -0.938, -0.941,
       -0.941, -0.961, -1.139, -1.17 , -0.959, -0.934, -0.932, -0.91 ,
       -0.783, -0.783, -0.493, -0.485, -0.485, -0.309, -0

#### .get()直接获取数据

In [419]:
###.get()操作直接获取数据，以dict字典的形式返回
entry.get("wall")

{'ids_properties': {'comment': {},
  'provider': 'Guo, Yong',
  'creation_date': '2020-10-12',
  'homogeneous_time': 2},
 'description_2d': {'type': {'name': 'equilibrium', 'index': 1},
  'limiter': {'type': {'name': 'limiter', 'index': 0},
   'unit': {'name': 'limiter',
    'closed': 1,
    'outline': {'r': array([2.277, 2.273, 2.267, 1.94 , 1.94 , 1.802, 1.773, 1.751, 1.736,
            1.714, 1.707, 1.696, 1.665, 1.656, 1.635, 1.612, 1.478, 1.459,
            1.44 , 1.436, 1.399, 1.379, 1.392, 1.43 , 1.439, 1.442, 1.437,
            1.363, 1.361, 1.361, 1.361, 1.363, 1.421, 1.423, 1.422, 1.418,
            1.331, 1.367, 1.564, 1.597, 1.598, 1.624, 1.754, 1.765, 1.814,
            1.824, 1.825, 1.841, 1.971, 1.971, 2.267, 2.273, 2.277, 2.277,
            2.306, 2.328, 2.343, 2.35 , 2.35 , 2.35 , 2.343, 2.328, 2.306,
            2.277]),
     'z': array([ 0.485,  0.485,  0.493,  0.809,  0.809,  0.926,  0.956,  0.993,
             1.033,  1.131,  1.162,  1.142,  1.117,  1.111,  1.096, 

#### SpDB中数据以字典形式在内存中直接交互

In [435]:
### 打印IDS magnetics中的关键keys
shot_num = 70754
time_slice = 10
DATA_PATH = "/scratch/jupytertest/workspace_fytok/fytok_data"
entry = open_entry(f"east+mdsplus://{DATA_PATH}/mdsplus/~t/?shot=70745")
entry.get("magnetics]").keys()


dict_keys(['ids_properties', 'b_field_pol_probe', 'flux_loop', 'ip'])

In [ ]:
#### SpDB中list的访问

In [436]:
### 例如，b_field_pol_probe探针有多个，以list形式存在
type(entry.get("magnetics")["b_field_pol_probe"])


list

In [437]:

### 每个探针下面是一个字典结构，获得其keys
entry.get("magnetics.b_field_pol_probe[0]").keys()


dict_keys(['name', 'identifier', 'position', 'toroidal_angle', 'field', '@id'])

In [438]:

entry.child("magnetics.b_field_pol_probe[0].position").__value__

[{'r': 1.2905, 'z': 0.0, '@id': '*'}]

#### SpDB中数据按照树状结构层层访问

In [439]:
### SpDB中数据按照层层路径进行访问:wall.description_2d[0].limiter.unit.outlin
### 获取outline的数据：
entry.get("wall.description_2d[0].limiter.unit.outline")

{'r': array([2.277, 2.273, 2.267, 1.94 , 1.94 , 1.802, 1.773, 1.751, 1.736,
        1.714, 1.707, 1.696, 1.665, 1.656, 1.635, 1.612, 1.478, 1.459,
        1.44 , 1.436, 1.399, 1.379, 1.392, 1.43 , 1.439, 1.442, 1.437,
        1.363, 1.361, 1.361, 1.361, 1.363, 1.421, 1.423, 1.422, 1.418,
        1.331, 1.367, 1.564, 1.597, 1.598, 1.624, 1.754, 1.765, 1.814,
        1.824, 1.825, 1.841, 1.971, 1.971, 2.267, 2.273, 2.277, 2.277,
        2.306, 2.328, 2.343, 2.35 , 2.35 , 2.35 , 2.343, 2.328, 2.306,
        2.277]),
 'z': array([ 0.485,  0.485,  0.493,  0.809,  0.809,  0.926,  0.956,  0.993,
         1.033,  1.131,  1.162,  1.142,  1.117,  1.111,  1.096,  1.084,
         1.025,  1.021,  1.024,  1.026,  1.039,  1.049,  1.014,  0.909,
         0.873,  0.835,  0.799,  0.456,  0.454,  0.   , -0.454, -0.456,
        -0.725, -0.748, -0.749, -0.77 , -1.011, -0.977, -0.938, -0.941,
        -0.941, -0.961, -1.139, -1.17 , -0.959, -0.934, -0.932, -0.91 ,
        -0.783, -0.783, -0.493, -0.485, -0.4